In [8]:
%load_ext autoreload
%autoreload 2
from finetune import *
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
args

{'cultural_corpus': 'yo-bm25-10000',
 'pretrained_model': 'FacebookAI/xlm-roberta-base',
 'corpus_chunk_size': 128,
 'wwm_probability': 0.15,
 'batch_size': 64}

In [10]:
tokenizer = mk_tokenizer(args)
mlm_model, mc_model = mk_models(args)

mask token id is 250001 and mask token is <mask>
tokenizer max len 512
model config: XLMRobertaConfig {
  "_name_or_path": "FacebookAI/xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mlm_model num param: 278295186
mc_model num param: 278044417


In [11]:
fill_mask(args, mlm_model, tokenizer, f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}.")

input: The cat said <mask>.
pred 0: The cat said _:_.
pred 1: The cat said _,_.
pred 2: The cat said _that_.
pred 3: The cat said _:_.
pred 4: The cat said _._.
pred 5: The cat said _it_.
pred 6: The cat said _he_.
pred 7: The cat said _"_.
pred 8: The cat said _to_.
pred 9: The cat said _..._.


In [12]:
toy_text = f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}."
toy_input = tokenizer(toy_text, return_tensors="pt")
mlm_model(**toy_input).keys()

odict_keys(['logits', 'hidden_states'])

In [13]:
lm_corpus, corpus_mask_collator, corpus_wwm_collator = mk_corpus(args, tokenizer)
lm_corpus

Map: 100%|██████████| 30/30 [00:00<00:00, 12162.10 examples/s]

index 0 train example:
{'input_ids': [0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 717], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 16
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1
    })
})

In [15]:
processed_figqa_datasets, figqa_data_collator = mk_figqa_dataset(args, tokenizer)
processed_figqa_datasets

Map: 100%|██████████| 2/2 [00:00<00:00, 370.65 examples/s]

figqa data looks like:
train: {'labels': 0, 'input_ids': [[0, 581, 7515, 2804, 163, 8770, 2, 2, 163, 8770, 2], [0, 581, 7515, 2804, 163, 8770, 2, 2, 4323, 4390, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> The cat said meow</s></s> meow</s>', '<s> The cat said meow</s></s> woof</s>']
val: {'labels': 0, 'input_ids': [[0, 581, 7515, 2804, 163, 8770, 2, 2, 163, 8770, 2], [0, 581, 7515, 2804, 163, 8770, 2, 2, 4323, 4390, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> The cat said meow</s></s> meow</s>', '<s> The cat said meow</s></s> woof</s>']
test: {'labels': 0, 'input_ids': [[0, 62, 45730, 7515, 7, 1884, 47, 3249, 83, 163, 8770, 2, 2, 163, 8770, 2], [0, 62, 45730, 7515, 7, 1884, 47, 3249, 83, 163, 8770, 2, 2, 4323, 4390, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> A 

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
})

## MLM train

In [8]:
training_args = TrainingArguments(
    output_dir=f"xlmr-finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    per_device_train_batch_size=args['batch_size'],
    per_device_eval_batch_size=args['batch_size'],
    logging_steps=1,
    num_train_epochs=10,
    use_cpu=True,
    use_mps_device=False,
)
trainer = Trainer(
    model=mlm_model,
    args=training_args,
    train_dataset=lm_corpus["train"],
    eval_dataset=lm_corpus["val"],
    data_collator=corpus_mask_collator,
    tokenizer=tokenizer,
)

/Users/chaosarium/anaconda3/envs/multi/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
trainer.evaluate()

100%|██████████| 1/1 [00:00<00:00, 5229.81it/s]


{'eval_loss': 12.164128303527832,
 'eval_runtime': 0.2334,
 'eval_samples_per_second': 4.284,
 'eval_steps_per_second': 4.284}

In [ ]:
trainer.train()

## FigQA training

In [16]:
train_dataset = processed_figqa_datasets["train"]
eval_dataset = processed_figqa_datasets["val"]

train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=figqa_data_collator, batch_size=args['batch_size'])
eval_dataloader = DataLoader(eval_dataset, collate_fn=figqa_data_collator, batch_size=args['batch_size'])

In [25]:
optimizer = torch.optim.AdamW(mc_model.parameters(), lr=2e-5)
lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=3,
    num_training_steps=30,
)

In [26]:
mc_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(mc_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler)

In [27]:
metric = evaluate.load("accuracy")

In [29]:
completed_steps = 0

for epoch in range(0, 10):
    mc_model.train()
    mc_model, total_loss, completed_steps = train_model(train_dataloader, mc_model, accelerator, optimizer, lr_scheduler, args, completed_steps, checkpointing_steps=500)
    eval_metric = eval_model(mc_model, eval_dataloader, metric, accelerator, epoch, args)

    print({
        "accuracy": eval_metric,
        "train_loss": total_loss.item() / len(train_dataloader),
        "epoch": epoch,
        "step": completed_steps,
    })

    # if args["push_to_hub"] and epoch < args["num_train_epochs"] - 1:
    #     accelerator.wait_for_everyone()
    #     unwrapped_model = accelerator.unwrap_model(mc_model)
    #     unwrapped_model.save_pretrained(
    #         args["output_dir"], is_main_process=accelerator.is_main_process, save_function=accelerator.save
    #     )
    #     if accelerator.is_main_process:
    #         tokenizer.save_pretrained(args["output_dir"])
    #         # repo.push_to_hub(
    #         #     commit_message=f"Training in progress epoch {epoch}", blocking=False, auto_lfs_prune=True
    #         # )

    # if args["checkpointing_steps"] == "epoch":
    #     output_dir = f"epoch_{epoch}"
    #     if args["output_dir"] is not None:
    #         output_dir = os.path.join(args["output_dir"], output_dir)
    #     accelerator.save_state(output_dir)

# if args["output_dir"] is not None:
#     accelerator.wait_for_everyone()
#     unwrapped_model = accelerator.unwrap_model(mc_model)
#     unwrapped_model.save_pretrained(
#         args["output_dir"], is_main_process=accelerator.is_main_process, save_function=accelerator.save
#     )
#     if accelerator.is_main_process:
#         tokenizer.save_pretrained(args["output_dir"])
#         # if args["push_to_hub"]:
#         #     repo.push_to_hub(commit_message="End of training", auto_lfs_prune=True)
#     with open(os.path.join(args["output_dir"], "all_results.json"), "w") as f:
#         json.dump({"eval_accuracy": eval_metric["accuracy"]}, f)

eval_metric["accuracy"]


100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


tensor(0.7454, grad_fn=<NllLossBackward0>)
epoch 0: {'accuracy': 0.5}
{'accuracy': {'accuracy': 0.5}, 'train_loss': 0.7454084157943726, 'epoch': 0, 'step': 1}


100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


tensor(0.7176, grad_fn=<NllLossBackward0>)
epoch 1: {'accuracy': 0.5}
{'accuracy': {'accuracy': 0.5}, 'train_loss': 0.7176364660263062, 'epoch': 1, 'step': 2}


100%|██████████| 1/1 [00:01<00:00,  1.47s/it]


...

100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


tensor(0.6159, grad_fn=<NllLossBackward0>)
epoch 8: {'accuracy': 0.5}
{'accuracy': {'accuracy': 0.5}, 'train_loss': 0.6159067153930664, 'epoch': 8, 'step': 9}


100%|██████████| 1/1 [00:01<00:00,  1.21s/it]

tensor(0.6657, grad_fn=<NllLossBackward0>)
epoch 9: {'accuracy': 0.5}
{'accuracy': {'accuracy': 0.5}, 'train_loss': 0.6657137274742126, 'epoch': 9, 'step': 10}


0.5